1. Cài đặt thư viện pandas

In [2]:
# pip install pandas

In [3]:
# pip install beautifulsoup4

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import requests as rq
import json
import os
import math
import datetime, time

In [24]:
# Set biến
input_file = '/home/xoanvt/DE_study_unigap_project/project02/products.xlsx'
output_dir = '/home/xoanvt/DE_study_unigap_project/project02/output'
log_error = Path("/home/xoanvt/DE_study_unigap_project/project02/log_error.txt")
url = "https://api.tiki.vn/product-detail/api/v1/products/"
parallel_nbr = 50
time_out = 10
file_sz = 1000
retry = 3

In [6]:
def clean_html_text(html_content): # Dùng để làm sạch đoạn mã html trong desciption 
    if not html_content:
        return ""
    
    soup = BeautifulSoup(html_content, "html.parser")
    
    text = soup.get_text(separator="\n", strip=True)
    
    return text


Tạo function truyền vào id của sản phẩm (product_id) có các bước sau:
- Tạo link API cho từng sản phẩm.
- Lấy thông tin sản phẩm qua API bằng phương thức GET
- return kết quả call API

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0 Safari/537.36",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7",
    "Referer": "https://tiki.vn/",
    "X-Requested-With": "XMLHttpRequest",
}

def fetch_product_data(session, product_id):
    for attempt in range(retry):
        try:
            api_url = f"{url}{product_id}"
            resp = session.get(api_url, headers=HEADERS, timeout=time_out)
            # resp = rq.get(api_url, headers=HEADERS, timeout=time_out)
            if resp.status_code == 403:
                raise Exception("403 Forbidden")
            resp.raise_for_status()
            data = resp.json()

            # Lấy các trường yêu cầu
            cleaned = {
                "id": data.get("id"),
                "name": data.get("name"),
                "url_key": data.get("url_key"),
                "price": data.get("price"),
                "description": clean_html_text(data.get("description")),
                # "images_url": data.get("images_url", [])
            }
            return {"product_id": product_id, "status": "success", "data": cleaned}
        
        # Bỏ qua sản phẩm lỗi và lưu ID lỗi vào file log_error
        except Exception as e:
            if attempt < retry - 1:
                    time.sleep(1)
                    continue
            with open(log_error, "a") as f:
                f.write(f"{product_id}\n")
            return {"product_id": product_id, "error": str(e)}

In [8]:
start_time = datetime.datetime.now()
print(start_time)

2025-08-15 01:11:43.120083


- Đọc file excel chuyển thành dataframe, sử dụng thư viện pandas
- Xóa các bản ghi null, bị trùng của cột "id"
- Chuyển giá trị cột "id" trong dataframe thành 1 list

In [9]:
df = pd.read_excel(input_file)
product_ids = df["id"].dropna().drop_duplicates().to_list()
# print(product_ids[:10])

In [11]:
# results = []

# with ThreadPoolExecutor(max_workers=parallel_nbr) as executor:
#     ft = {executor.submit(fetch_product_data, pid): pid for pid in product_ids[:10]}
#     for f in as_completed(ft):
#         results = f.result()
#         print(results)


Chạy test API với 10 sản phẩm đầu tiên. Gặp lỗi 403:

{'product_id': 154155413, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/154155413'}  
{'product_id': 1391347, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/1391347'}  
{'product_id': 179970479, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/179970479'}  
{'product_id': 197334787, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/197334787'}  
{'product_id': 214009046, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/214009046'}  
{'product_id': 74897599, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/74897599'}  
{'product_id': 253117062, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/253117062'}  
{'product_id': 130978358, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/130978358'}  
{'product_id': 171618108, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/171618108'}  
{'product_id': 139457837, 'error': '403 Client Error: Forbidden for url: https://api.tiki.vn/product-detail/api/v1/products/139457837'} 

In [ ]:
# Sử dụng requests.Session() để giữ cookie, headers giống Postman
session = rq.Session()

results = []
# results_status = []
file_count = 1

#  Dùng ThreadPoolExecutor để chạy parallel nhiều luồng call API

with ThreadPoolExecutor(max_workers=parallel_nbr) as executor:
    ft = {executor.submit(fetch_product_data, session, pid): pid for pid in product_ids}
    for f in as_completed(ft):
        result = f.result()
        results.append(result)

        #  Log trạng thái xử lý
        if len(results) % 10000 == 0:
            print(f"Đã xử lý {len(results)}/{len(product_ids)} sản phẩm")

# Thời gian call API xong 200k sản phẩm: 56m 23.9s

Đã xử lý 10000/200000 sản phẩm
Đã xử lý 20000/200000 sản phẩm
Đã xử lý 30000/200000 sản phẩm
Đã xử lý 40000/200000 sản phẩm
Đã xử lý 50000/200000 sản phẩm
Đã xử lý 60000/200000 sản phẩm
Đã xử lý 70000/200000 sản phẩm
Đã xử lý 80000/200000 sản phẩm
Đã xử lý 90000/200000 sản phẩm
Đã xử lý 100000/200000 sản phẩm
Đã xử lý 110000/200000 sản phẩm
Đã xử lý 120000/200000 sản phẩm
Đã xử lý 130000/200000 sản phẩm
Đã xử lý 140000/200000 sản phẩm
Đã xử lý 150000/200000 sản phẩm
Đã xử lý 160000/200000 sản phẩm
Đã xử lý 170000/200000 sản phẩm
Đã xử lý 180000/200000 sản phẩm
Đã xử lý 190000/200000 sản phẩm
Đã xử lý 200000/200000 sản phẩm


In [ ]:
#  Clean lại kết quả, chỉ lấy các kết quả chứa nội dung
results_avai = []
for r in results:    
    if r.get("status") == "success" and "data" in r:
        results_avai.append(r["data"])
    else:
        # lưu vào file log
        with open(log_error, "a", encoding="utf-8") as f:
            product_id = r.get("id", "unknown_id")
            f.write(f"{product_id}\n")
print(results_avai[:10])

[{'id': 206303845, 'name': 'Đồ chơi lắp ghép nhiều chủ đề ngành nghề - bác sĩ, nấu ăn, chăm sóc thú cưng, trang điểm', 'url_key': 'do-choi-huong-nghiep-nhieu-nganh-nghe-bac-si-nau-an-cham-soc-thu-cung-trang-diem-p206303845', 'price': 213000, 'description': 'THÔNG TIN SẢN PHẨM\nKích thước: 20 x 23 x 11 cm\nKhối lượng: 400g\nĐộ tuổi: 3+\nChất liệu: Nhựa ABS & PP\nPin: không pin\nXuất xứ: Trung Quốc\nHƯỚNG DẪN SỬ DỤNG\nLắp các chi tiết thành bộ hoàn chỉnh.\nSản phẩm sử dụng bằng tay – an toàn cho trẻ khi chơi.\nƯU ĐIỂM SẢN PHẨM\nNhiều kịch bản mô phỏng, trải nghiệm thú vị cho trẻ khi chơi.\nKiểu dáng đẹp mắt và chất liệu an toàn cho bé.\nTập cho bé khả năng thực hành, rèn trí tưởng tượng và khả năng thị giác.\nBé có khả năng tự thể hiện bản thân khi chơi.\nGiúp gia tăng tình cảm giữa cha mẹ và con khi chơi.\nSản phẩm giúp trẻ phát triển:\n- Khả năng quan sát, nhận biết màu sắc, vật thể.\n- Phát triển khả năng vận động của trẻ.\n- Phát triển khả năng giao tiếp với gia đình, bạn bè cùng chơ

In [ ]:
#  Nhóm kết quả, trả về các file json
total_files = math.ceil(len(results_avai) / file_sz)
for i in range(total_files):
    start_idx = i * file_sz
    end_idx = start_idx + file_sz
    batch_data = results_avai[start_idx:end_idx] # Cắt kết quả call API thành các list chưa data của 100 sản phẩm
    file_path = Path(output_dir) / f"products_batch_{i+1}.json"
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(batch_data, f, ensure_ascii=False, indent=2)
print(f"Hoàn thành! Tạo {total_files} file JSON")

Hoàn thành! Tạo 199 file JSON


Sau khi kiểm tra file Log_error, nhận thấy có một số id trả về unknown -> clean lại và lưu lại lỗi với các id lỗi.

In [25]:
clean_log_path = Path("log_error_clean.txt")

# Đọc file log gốc
with open(log_error, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Loại bỏ trống, unknown_id, None, và duplicates
clean_ids = sorted({line.strip() for line in lines if line.strip() and line.strip().lower() != "unknown_id"})

# Ghi lại vào file mới
with open(clean_log_path, "w", encoding="utf-8") as f:
    for pid in clean_ids:
        f.write(f"{pid}\n")

print(f"Đã xử lý xong. {len(clean_ids)} ID hợp lệ lưu vào {clean_log_path}")

Đã xử lý xong. 24029 ID hợp lệ lưu vào log_error_clean.txt


In [22]:
end_time = datetime.datetime.now()
print(end_time)

2025-08-15 05:30:28.586205
